In [45]:
import Pkg
# Pkg.add("BSplineKit")
Pkg.add("Dierckx")
Pkg.add("Convex")
Pkg.add("FiniteDiff")
Pkg.add("ForwardDiff")
Pkg.add("Interpolations")
Pkg.add("StaticArrays")
Pkg.add("GeometryBasics")
Pkg.activate(@__DIR__)

import FiniteDiff
import ForwardDiff as FD
import Convex as cvx
import ECOS
# using BSplineKit
using Dierckx
using Interpolations
using LinearAlgebra
using Plots
using Random
using JLD2
using Test
using MeshCat
const mc = MeshCat
using StaticArrays
using Printf
using GeometryBasics

   Resolving package versions...
  No Changes to `~/16745/final_proj/Project.toml`
  No Changes to `~/16745/final_proj/Manifest.toml`
   Resolving package versions...
  No Changes to `~/16745/final_proj/Project.toml`
  No Changes to `~/16745/final_proj/Manifest.toml`
   Resolving package versions...
  No Changes to `~/16745/final_proj/Project.toml`
  No Changes to `~/16745/final_proj/Manifest.toml`
   Resolving package versions...
  No Changes to `~/16745/final_proj/Project.toml`
  No Changes to `~/16745/final_proj/Manifest.toml`
   Resolving package versions...
  No Changes to `~/16745/final_proj/Project.toml`
  No Changes to `~/16745/final_proj/Manifest.toml`
   Resolving package versions...
  No Changes to `~/16745/final_proj/Project.toml`
  No Changes to `~/16745/final_proj/Manifest.toml`
   Resolving package versions...
  No Changes to `~/16745/final_proj/Project.toml`
  No Changes to `~/16745/final_proj/Manifest.toml`
  Activating project at `~/16745/final_proj`


In [144]:
include(joinpath(@__DIR__, "utils","fmincon.jl"))

fmincon (generic function with 1 method)

In [2]:
versioninfo()

Julia Version 1.6.7
Commit 3b76b25b64 (2022-07-19 15:11 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin21.4.0)
  CPU: Apple M2
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, westmere)


In [237]:
function dynamics(model::NamedTuple, x::Vector, u::Vector)::Vector
    # kinematic bicycle model continuous time dynamics
    # lr = distance from rear axle to COM
    # L = wheelbase

    lr, L = model.lr, model.L

    # along track, cross track, heading w.r.t track, steering angle, speed
    s, r, e_psi, δ, v = x

    # rate of steer, acceleration
    δdot, a = u

    # side slip angle
    β = atan(lr/L * tan(δ))

    # curvature of reftraj
    Tdot = reftraj_2nd_deriv(get_idx_from_dist(s)) / norm(reftraj_deriv(get_idx_from_dist(s)))
    κ = norm(Tdot)


    sin_e_psi, cos_e_psi = sincos(e_psi + β)

    sdot = v * cos_e_psi / (1 - r * κ)
    rdot = v * sin_e_psi

    psi_dot = v/lr * sin(β)

    # side slip angle rate + heading rate
    e_psi_dot = psi_dot - sdot * κ

    xdot = [
        sdot,
        rdot,
        e_psi_dot,
        δdot,
        a
    ]
    return xdot
end

function rk4(model::NamedTuple, ode::Function, x::Vector, u::Vector, dt::Real)::Vector
    k1 = dt * ode(model, x,        u)
    k2 = dt * ode(model, x + k1/2, u)
    k3 = dt * ode(model, x + k2/2, u)
    k4 = dt * ode(model, x + k3,   u)
    return x + (1/6)*(k1 + 2*k2 + 2*k3 + k4)
end

function hermite_simpson(model::NamedTuple, ode::Function, x1::Vector, x2::Vector, u::Vector, dt::Float64)::Vector
    ẋ1 = ode(model, x1, u)
    ẋ2 = ode(model, x2, u)
    x_half = 0.5 * (x1 + x2) + dt/8 * (ẋ1 - ẋ2)
    ẋ_half = ode(model, x_half, u)
    return x1 + dt/6 * (ẋ1 + 4 * ẋ_half + ẋ2) - x2
end

hermite_simpson (generic function with 2 methods)

In [238]:
X_mat = [0.0 0; 1 0; 2 0; 5 1; 10 2; 15 4; 25 5; 30 5]
X = [X_mat[i,:] for i in 1:size(X_mat,1)]

itp = Interpolations.extrapolate(
    Interpolations.interpolate(
        X_mat,
        (
            BSpline(Cubic(Interpolations.Natural(OnGrid()))),
            NoInterp()
        )
    ),
    Interpolations.Line()
)

# i is an index into the original data, where non-integer value i's
# produce interpolated data
function reftraj(i)
    return [itp(i, 1); itp(i, 2)]
end

function reftraj_deriv(i::Real)
    return FD.derivative(reftraj, i)
end

function reftraj_2nd_deriv(i::Real)
    return FD.derivative(reftraj_deriv, i)
end


"""
generate a vector of distances along the reftraj,
returns the vector of distances and the associated vector of indices
"""
function generate_distances()
    dt = 0.01
    ts = 0.0:dt:size(X, 1)
    drdts = copy(reftraj_deriv.(ts))
    print("drdts shape", size(drdts))
    drdts = [drdts[i, :] for i in 1:size(drdts, 1)]
    speeds = norm.(eachrow(drdts))
    distances = zeros(size(speeds, 1))
    distances[1] = speeds[1] * dt
    for i = 2:size(drdts, 1)
        distances[i] = speeds[i] * dt + distances[i - 1]
    end

    return distances, ts
end

distances, ts = generate_distances()
print("size of dist", size(distances), "size of ts", size(ts))

# 1-indexed, so add 1 since ts are 0-indexed
function get_idx_from_dist(_d)
    return 1 + linear_interpolation(
        distances,
        ts,
        extrapolation_bc=Interpolations.Line())(_d)
end

function get_dist_from_idx(_i)
    return linear_interpolation(
        ts,
        distances,
        extrapolation_bc=Interpolations.Line())(_i - 1)
end

function get_pos_from_dist(_d)
    return reftraj(get_idx_from_dist(_d))
end

function get_dist_from_pos(x)
    # compute distance to each knot pt
    dists_to_path = norm.(X .- Ref(x))
    knot_idx = argmin(dists_to_path)
    # subsample the distance function in the interval [knot idx - 1, knot idx + 1]
    # for each subsample along the path, compute distance from x  to subsample
    path_subsample = reftraj.(knot_idx-1:0.1:knot_idx+1)
    dists_to_subsample = norm.(path_subsample .- Ref(x))
    knot_idx_subsample = argmin(dists_to_subsample) * 0.1 + (knot_idx - 1)
    # @show "subsample range", knot_idx-1:0.01:knot_idx+1
    # @show "spline position along subsample range", path_subsample
    # @show "subsampled dists" dists_to_subsample
    # @show "closest idx", knot_idx
    # @show "closest subsample idx", knot_idx_subsample
    return get_dist_from_idx(knot_idx_subsample)
end

function matrix_to_vec(mat)
    vec = [mat[i,:] for i in 1:size(mat,1)]
    return vec
end


drdts shape(801,)size of dist(801,)size of ts(801,)

matrix_to_vec (generic function with 1 method)

In [254]:
# adapted from HW4
function create_idx(nx,nu,N)
    # create idx for indexing convenience
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    # and stacked dynamics constraints of size nx are
    # c[idx.c[i]] = <dynamics constraint at time step i>

    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]

    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx

    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x= x,u = u,c = c)
end

# function dynamics_constraints(params, Z)
#     idx = params.idx
#     N = params.N
#     c = zeros(eltype(Z), idx.nc)
#     model = params.model
#     dt = params.dt

#     for i = 1:N-1
#         xi = Z[idx.x[i]]
#         xi1 = Z[idx.x[i + 1]]
#         ui = Z[idx.u[i]]
#         c[idx.c[i]] = rk4(model, dynamics, xi, ui, dt) - xi1
#     end

#     return c
# end

function dynamics_constraints(params, Z)
    idx = params.idx
    N = params.N
    c = zeros(eltype(Z), idx.nc)
    model = params.model
    dt = params.dt

    for i = 1:N-1
        xi = Z[idx.x[i]]
        xi1 = Z[idx.x[i + 1]]
        ui = Z[idx.u[i]]
        c[idx.c[i]] = hermite_simpson(model, dynamics, xi, xi1, ui, dt)
    end

    return c
end

function equality_constraints(params, Z)
    idx, xi = params.idx, params.xi
    init_constraint = xi - Z[idx.x[1]]
    return [init_constraint; dynamics_constraints(params, Z)]
end

function inequality_constraints(params, Z)
    return [0]
end

function cost(params::NamedTuple, Z::Vector)::Real
    # cost function
    idx, N, s_goal = params.idx, params.N, params.s_goal
    Q, R = params.Q, params.R

    J = 0
    for i = 1:N-1
        Xe = Z[idx.x[i]] - [s_goal, 0, 0, 0, 0]
        U = Z[idx.u[i]]
        J += Xe' * Q * Xe + U' * R * U
    end

    Xfe = Z[idx.x[N]] - [s_goal, 0, 0, 0, 0]
    J += Xfe' * Q * Xfe

    return 0.5 * J
end

function optimize(Xpath)
    dt = 0.2 #s
    t_horizon = 10 #s
    N = Int(t_horizon / dt)

    model = (L = 1.2,
        lr = 0.4,
        reftraj_2nd_deriv=reftraj_2nd_deriv,
        reftraj_deriv=reftraj_deriv,
        reftraj=reftraj)

    track = (width = 3.0)
    idx = create_idx(5, 2, N)

    # mostly care about progress
    Q = diagm([100; 0; 1.0; 1.0; 0])
    R = diagm([0.1; 0.1])

    @show length(Xpath)
    pathlen = get_dist_from_idx(length(Xpath))
    @show pathlen
    s_goal = pathlen * 1.5

    @show "Xpath", Xpath[1]
    si = get_dist_from_pos(Xpath[1])

    print(get_pos_from_dist(si), Xpath[1])
    # get pos from dist returns N x 2
    # N is 1 here so still need to index into the return value
    pos_to_ref = get_pos_from_dist(si) - Xpath[1]

    # check sign via cross product with tangent
    tangent = reftraj_deriv(1)
    @show size(tangent)
    cross_product = (tangent[1] * pos_to_ref[2]) - (tangent[2] * pos_to_ref[1])
    r_norm = norm(pos_to_ref)

    # TODO: figure out sign convention
    ri = sign(cross_product) * r_norm
    xi = [si, ri, 0, 0, 0]
    @show ("xi: ", xi)

    params = (
        model=model,
        track=track,
        idx=idx,
        N=N,
        Q=Q,
        R=R,
        dt=dt,
        xi=xi,
        s_goal=s_goal)


    z0 = zeros(idx.nz)
    # TODO: if this isn't good enough, initialize steering angle using curvature of reference

    # initialize velocity by assuming constant velocity and that we reach the end of the track
    # at the end of the horizon

    # since we initialize the state with reftraj, e_psi is always 0. Since we assume const vel,
    # v is always path len / time horizon.

    v0 = 13.0
    ts = 0.0:dt:t_horizon
    dists = ts .* v0
    idxs = get_idx_from_dist.(ts)

    # N x 2, where each row is (s, r)
    sr_0 = reftraj.(idxs)

    for i = 1:N
        z0[idx.x[i]] = [sr_0[i][1], sr_0[i][2], 0.0, 0.0, v0]
    end

    # TODO: primal bounds

    x_l = -Inf*ones(idx.nz) # update this
    x_u =  Inf*ones(idx.nz) # update this

    for i = 1:N-1
        # cross track limits
        x_l[idx.x[i+1][2]] = -3.0
        x_u[idx.x[i+1][2]] = 3.0

        # steering angle limits
        x_l[idx.x[i+1][4]] = -pi/6
        x_u[idx.x[i+1][4]] = pi/6

        # speed limits
        x_l[idx.x[i+1][5]] = 13.0
        x_u[idx.x[i+1][5]] = 13.0

        # steering rate limits
        x_l[idx.u[i][1]] = -2 * pi
        x_u[idx.u[i][1]] = 2 * pi

        # # accelerate/brake limits
        # x_l[idx.u[i][2]] = 0.0
        # x_u[idx.u[i][2]] = 0.0

    end

    # inequality constraint bounds
    c_l =  -Inf*ones(1)
    c_u =   Inf*ones(1)
    diff_type = :auto

    Z = fmincon(cost,equality_constraints,inequality_constraints,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-6, c_tol = 1e-6, max_iters = 500, verbose = true)

    @show "dynamics constraint", dynamics_constraints(params, Z)

    for i = 1:N-1
        print(i, "state:", Z[idx.x[i]])
        print("ctrl:", Z[idx.u[i]])
        print("\n")
    end
    print("final state", Z[idx.x[N]])
    print("\n")
end

optimize(X)

length(Xpath) = 8
pathlen = 26.905536915048977
("Xpath", Xpath[1]) = ("Xpath", [0.0, 0.0])
[0.10955650979045006, 0.007720027481964966][0.0, 0.0]size(tangent) = (2,)
("xi: ", xi) = ("xi: ", [0.12092296620497843, -0.10982817335177379, 0.0, 0.0, 0.0])
---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :auto (ForwardDiff.jl)----
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------
This is Ipopt version 3.14.14, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:    74750
Number of nonzeros in inequality constraint Jacobian.:      299
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      299
                     variables with only lower boun